In [33]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pickle as pk
import resources.RES as res
from sklearn.preprocessing import minmax_scale as sk_minmax

In [34]:
# retrieve context, maximum extents, and maximum extends as voxelised volume

context_path = os.path.relpath("../data/movedcontext.obj") 
context_mesh = tm.load(context_path)

# extents_path = os.path.relpath("../data/maximumextents.obj") 
# extents_mesh = tm.load(extents_path)

env_lat_path = os.path.relpath("../data/macrovoxels.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

In [35]:
# skydome
skydome = pk.load(open("../data/skydome.pk", "rb"))

In [36]:
# finding the test points on the surrounding area/street around the building
edges = np.zeros((7,7,7)) # TODO: need to change
edges[1] = np.pad(np.zeros((5,5)), 1, constant_values=1)

# creating the lattice for the test point origins - basically a padding of the original envelope
streetedges = tg.to_lattice(edges,envelope_lattice.minbound - envelope_lattice.unit, unit=envelope_lattice.unit).T

In [37]:
# create trimesh cuboids for computing intersections
def transform_mat(value):
    mat = np.identity(4)
    mat[:3,-1] = np.array(value)
    return mat

In [38]:
# voxel cuboid meshes
vox_cuboids = [tm.creation.box(envelope_lattice.unit, transform=transform_mat(ct)) for ct in envelope_lattice.centroids]

# number of faces per voxel
f_numb = len(vox_cuboids[0].faces)

# combine voxels into one mesh
combined_voxels = tm.util.concatenate(vox_cuboids)

In [39]:
# combine voxels and context into one mesh
combined_meshes = tm.util.concatenate(combined_voxels, context_mesh)

# shoot towards the skydome points from all of the voxels
SVF_ray_ctr = np.tile(skydome, [len(streetedges.centroids),1]) # sky view factor ray for each centroid of the lattice
SVF_ctr_ray = np.tile(streetedges.centroids, [1, len(skydome)]).reshape(-1, 3) # sky view factor centroid for each ray

In [40]:
# intersection of rays from ground centroids to sky patches with all voxel faces
face_id, ray_id = combined_meshes.ray.intersects_id(ray_origins=SVF_ctr_ray, ray_directions=SVF_ray_ctr, multiple_hits=True) # environment too complex? Takes about 70s

In [41]:
G1, U1  = res.ground_graph(skydome, face_id, ray_id, envelope_lattice, f_numb, streetedges)

In [42]:
# save interedependencies for later use
pk.dump(G1, open("../data/SvFG1.pk", "wb"))

# save obstructions for later use
pk.dump(U1, open("../data/SvFU1.pk", "wb"))

In [43]:
# number of rays shot from each test point at street level
skydome_rays = np.full(len(streetedges.centroids),len(skydome)) 

# number of rays that hit the context from each test point at street level
context_hits = np.sum(U1,axis=1) 

# how much sky patches are visible from each test point at starting situation
sky_visibility = skydome_rays - context_hits 

# percentage of sky visible from the test points at starting situation -> value/weights of the test points
vis_percent = sky_visibility/skydome_rays 

In [44]:
# sum per row twice
sky_obstructed = (G1.sum(axis=1)).sum(axis=1) # number of blocks of sky visibility from street level by each voxel

In [45]:
def reshape_and_store_to_lattice(values_list, envelope_lattice):
    env_all_vox_id = envelope_lattice.indices.flatten()
    env_all_vox = envelope_lattice.flatten() # envelope inclusion condition: True-False
    env_in_vox_id = env_all_vox_id[env_all_vox] # keep in-envelope voxels (True)

    # initialize array
    values_array = np.full(env_all_vox.shape, 0.0)
    
    # store values for the in-envelope voxels
    values_array[env_in_vox_id] = values_list

    # reshape to lattice shape
    values_array_3d = values_array.reshape(envelope_lattice.shape)

    # convert to lattice
    values_lattice = tg.to_lattice(values_array_3d, envelope_lattice)

    return values_lattice

In [46]:
# apply weight to ground test points (currently percentage of sky visible in default situation) - blocking bright spaces will have high costs and vice versa
w1 = vis_percent

# dot product with ray weights
o1 = np.array(np.dot(G1.sum(axis=2),w1), dtype='int64') # sun blocking

# normalize values
c4_norm = sk_minmax(o1) # voxel sky view factor potential TODO: need to verify whether values should be high or low at the top of the lattice

# reshape and store to lattice
street_lat = reshape_and_store_to_lattice(c4_norm, envelope_lattice)

In [47]:
# save normalized values for later use
pk.dump(c4_norm, open("../data/svf_norm.pk", "wb"))
# normalized values of sky blocking/sky blocked cost of each voxel

In [48]:
# visualize sky view factor values 
p = pv.Plotter(notebook=True)

def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# fast visualization of the lattice
# envelope_lattice.fast_vis(p)

# add context
# p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')

base_lattice = street_lat
grid = pv.UniformGrid() # Create the spatial reference
grid.dimensions = np.array(base_lattice.shape) + 1 # Set the grid dimensions
grid.origin = base_lattice.minbound - 0.5 * base_lattice.unit # The bottom left corner of the data set
grid.spacing = base_lattice.unit # These are the cell sizes along each axis

# Add the data values to the cell data
# grid.point_arrays["Score"] = base_lattice.flatten(order="F")  # Flatten the Lattice
grid.cell_arrays["Score"] = base_lattice.flatten(order="F")  # Flatten the Lattice
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.0
p.add_volume(grid, cmap="coolwarm", clim=[base_lattice.min(), base_lattice.max()],opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(204.88887394336027, 84.88887394336027, 144.88887394336027),
 (60.0, -60.0, 0.0),
 (0.0, 0.0, 1.0)]